<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemHYstanzaWithDemoV202511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Armenian lemmatization with Stanza

Version 2025 09 - adjusted for Colab update with PyTorch >2.5 (with the Torch version downgrading route selected).

## downloading evaluation sets
- 420 words: test with about 420 words of Armenian text
- Armenian "Brown-type" corpus b

In [ ]:
# download some sample Armenian sentences, or upload your own, change the
!wget https://heibox.uni-heidelberg.de/f/ce6096da570f47b99500/?dl=1
!mv index.html?dl=1 evaluation-set-v01.txt
!wget https://heibox.uni-heidelberg.de/f/a847a12bffd4491f9070/?dl=1
!mv index.html?dl=1 TED2020-dehy-hy-aa.txt


!wget https://heibox.uni-heidelberg.de/f/7a091a3c0372428a9aa2/?dl=1
!mv index.html?dl=1 udhr-all-languages.zip
!unzip udhr-all-languages.zip

In [ ]:
%%bash

# Downloading UDHR
# !wget https://unicode.org/udhr/assemblies/udhr_txt.zip
# Alternatively, downloading from the UN website, converting pdf to txt
# the link aboout pdf2txt conversion: https://chatgpt.com/share/68bbeed8-0d08-800e-8418-b0816c6393d4
# !rm --recursive udhr-all-languages

# delete lines which are not translations in some files (hy)
# delete between lines $a and $b inclusive
a=9
b=21
awk -v m=$a -v n=$b 'm <= NR && NR <= n {next} {print}' < /content/udhr-all-languages/udhr_hye.txt >/content/udhr-all-languages/udhr_hye_v02.txt

a=1
b=6
awk -v m=$a -v n=$b 'm <= NR && NR <= n {next} {print}' < /content/udhr-all-languages/udhr_hye_v02.txt >/content/udhr-all-languages/udhr_hye_v03.txt



# put paragraph tags
# awk '{print "<p>\n"$0 ; print "</p>"}' udhr/udhr_hye2.txt >udhr/udhr_hye_v03.txt


cp /content/udhr-all-languages/udhr_hye_v03.txt /content/udhr_hye_v03.txt
head --lines=10 /content/udhr_hye_v03.txt

ՄԱՐԴՈՒ ԻՐԱՎՈՒՆՔՆԵՐԻ ՀԱՄԸՆԴՀԱՆՈՒՐ ՀՌՉԱԿԱԳԻՐ
    ՄԻԱՎՈՐՎԱԾ ԱԶԳԵՐԻ ԿԱԶՄԱԿԵՐՊՈՒԹՅՈՒՆ
    ՆԵՐԱԾԱԿԱՆ
    Քանզի մարդկային ընտանիքի բոլոր անդամներին ներհատուկ արժանապատվությունը և հավասար ու անօտարելի իրավունքները աշխարհի ազատության, արդարության ու խաղաղության հիմքն են․
    Քանզի մարդու իրավունքների նկատմամբ քամահրանքն ու արհամարհանքը հանգեցրել են մարդկության խիղճը խռոված բարբարոսական գործողությունների, և քանի որ այնպիսի աշխարհի ստեղծումը, ուր մարդիկ կվայելեն խոսքի ու համոզմունքների ազատություն և զերծ կլինեն վախից ու կարիքից հռչակվել է որպես մարդկանց բարձրագույն ձգտում․
    Քանզի անհրաժեշտ է, որպեսզի մարդը, որպես մի վերջին միջոցի, չդիմի ապստամբության ընդդեմ բռնության ու ճնշման, օրենքի իշխանությամբ պաշտպանել մարդու իրավունքները․
    Քանզի անհրաժեշտ է նպաստել ազգերի միջև բարեկամական հարաբերությունների զարգացմանը․
    Քանզի Միավորված ազգերի ժողովուրդները կանոնադրության մեջ վերահավաստել են իրենց հավատը մարդու հիմնական իրավունքների, անձի արժանապատվության ու արժեքի, տղամարդու ու կնոջ հավասար իրավուն

## Installing stanza

### Explanation:

It is important to downgrade to Torch 2.5, because Stanza doesn't work with later versions without setting some non-default parameters. See the ChatGPT explanation at: https://chatgpt.com/share/68bad9f0-fc38-800e-a17d-898e6fef0e35

This will take some time (around 4 min).

You can ignore the error message ``` ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts. ``` The script is still working.

Note: After running, accept re-launch of the Runtime, if it is requested!




In [ ]:
# it is important to downgrade to Torch 2.5, because Stanza doesn't work with later versions. See the ChatGPT explanation at https://chatgpt.com/share/68bad9f0-fc38-800e-a17d-898e6fef0e35
!pip install spacy-stanza
!pip install torch==2.5.1

In [ ]:
import stanza
import spacy_stanza
import torch
print("Torch version:", torch.__version__)
# this should be "Torch version: 2.5.1+cu124" for Stanza to work!

Torch version: 2.5.1+cu124


### testing English stanza (optional)

In [ ]:
# optional
# Download the stanza model if necessary
stanza.download("en")

# Initialize the pipeline
nlp = spacy_stanza.load_pipeline("en")

doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)
print(doc.ents)

### Downloading and testing Armenian stanza

In [ ]:
stanza.download("hy")
# Load Armenian pipeline (make sure stanza models are downloaded first)
nlp_hy = spacy_stanza.load_pipeline("hy")

In [ ]:
# optional
# Example Armenian text
text = "Ես գնում եմ դպրոց։"
doc = nlp_hy(text)

for token in doc:
    print(f"\n=== Token: {token.text} ===")

    # Core text info
    print("Text:        ", token.text)
    print("Lemma:       ", token.lemma_)
    print("POS (UPOS):  ", token.pos_)
    print("Tag (XPOS):  ", token.tag_)
    print("Morph:       ", token.morph)
    print("Dep:         ", token.dep_)
    print("Head:        ", token.head.text)
    print("HeadI:       ", token.head.i)

    # Entity info
    print("Ent type:    ", token.ent_type_)
    print("Ent IOB:     ", token.ent_iob_)

    # Document & sentence info
    print("Is sent start:", token.is_sent_start)
    print("Sentence:    ", token.sent.text)

    # Orthographic info
    print("Lower:       ", token.lower_)
    print("Shape:       ", token.shape_)
    print("Whitespace:  ", repr(token.whitespace_))

    # Booleans
    print("is_alpha:    ", token.is_alpha)
    print("is_digit:    ", token.is_digit)
    print("is_punct:    ", token.is_punct)
    print("is_space:    ", token.is_space)
    print("is_stop:     ", token.is_stop)

    # Position info
    print("Index:       ", token.i)
    print("Char offset: ", token.idx)


In [ ]:
### optional
doc = nlp_hy("ՄԱՐԴՈՒ ԻՐԱՎՈՒՆՔՆԵՐԻ ՀԱՄԸՆԴՀԱՆՈՒՐ ՀՌՉԱԿԱԳԻՐ. ՆԵՐԱԾԱԿԱՆ. Քանզի մարդկային ընտանիքի բոլոր անդամներին ներհատուկ արժանապատվությունըև հավասար ու անօտարելի իրավունքները աշխարհի ազատության, արդարության ու խաղաղության հիմքն են.")

In [ ]:
### optional
for token in doc:
    # print(token.i, token.text, token.lemma_, token.pos_, token.morph, token.dep_, token.head.i, token.head.text)
    print(token.text, token.pos_, token.lemma_)


ՄԱՐԴՈՒ NOUN մարդ
ԻՐԱՎՈՒՆՔՆԵՐԻ ADJ իրավունքների
ՀԱՄԸՆԴՀԱՆՈՒՐ ADJ համընդհանուր
ՀՌՉԱԿԱԳԻՐ NOUN հայրանահ
. PUNCT .
ՆԵՐԱԾԱԿԱՆ NOUN ներախական
. PUNCT .
Քանզի SCONJ քանզի
մարդկային ADJ մարդկային
ընտանիքի NOUN ընտանիք
բոլոր DET բոլոր
անդամներին NOUN անդամ
ներհատուկ ADJ ներհատուկ
արժանապատվությունը NOUN արժանապատվություն
և CCONJ և
հավասար ADJ հավասար
ու CCONJ ու
անօտարելի ADJ անօտարելի
իրավունքները NOUN իրավունք
աշխարհի NOUN աշխարհ
ազատության NOUN ազատություն
, PUNCT ,
արդարության NOUN արդարություն
ու CCONJ ու
խաղաղության NOUN խաղաղություն
հիմքն NOUN հիմք
են AUX եմ
. PUNCT .


### full analysis of the file (optional)
- includes dependency parsing

Note: a warning about NER possible: ``` /usr/local/lib/python3.12/dist-packages/spacy/language.py:1041: UserWarning: Can't set named entities because of multi-word token expansion  ```, just ingnore it, see the following explanation for more details: https://chatgpt.com/share/68baeaad-03bc-800e-a29c-e90f7a028996

In [ ]:
### optional
with open('/content/TED2020-dehy-hy-aa.txt', 'r', encoding='utf-8') as infile, open('/content/TED2020-dehy-hy-aa-ANALYSIS-full-v01.txt', 'w') as outfile:
    # read sample.txt an and write its content into sample2.txt
    # outfile.write("{token.i}\t{token.text}\t{token.lemma_}\t{token.pos_}\t{token.morph}\t{token.dep_}\t{token.head.i}\t{token.head.text}\t{parentLem}\t{SLAncestors}\n")
    outfile.write("{token.text}\t{token.pos_}\t{token.lemma_}\n")

    j=0
    for line in infile:
        line = line.strip()
        j = j+1
        if (j % 20 == 0): print(str(j), line)

        doc = nlp_hy(line)
        # outfile.write(line + '\n')

        for token in doc:
            LAncestors = list(token.ancestors)

            # print(str(LAncestors))
            try:
                SLAncestors = str(list(token.ancestors))
                parent = LAncestors[0]
                parentLem = parent.lemma_
            except:
                parentLem = "NONE"
            # outfile.write(f"{token.i}\t{token.text}\t{token.lemma_}\t{token.pos_}\t{token.morph}\t{token.dep_}\t{token.head.i}\t{token.head.text}\t{parentLem}\t{SLAncestors}\n")
            outfile.write(f"{token.text}\t{token.pos_}\t{token.lemma_}\n")


### function for lemmatization

In [ ]:
def parseFile(iFileName, oFileName, nlp_model = nlp_hy):
    with open(iFileName, 'r', encoding='utf-8') as infile, open(oFileName, 'w') as outfile:
        # read sample.txt an and write its content into sample2.txt
        # write the name of the fields (does not change actual fields printed, see last comment below)
        # outfile.write("{token.text}\t{token.pos_}\t{token.morph}\t{token.lemma_}\n")
        outfile.write("{token.text}\t{token.pos_}\t{token.lemma_}\n")
        c = 0
        for line in infile:
            c+=1
            if c%20 == 0: print('line no. ' + str(c))
            line = line.strip()
            doc = nlp_model(line)
            # outfile.write(line + '\n')
            for token in doc:
                LAncestors = list(token.ancestors)
                # print(str(LAncestors))
                try:
                    SLAncestors = str(list(token.ancestors))
                    parent = LAncestors[0]
                    parentLem = parent.lemma_
                except:
                    parentLem = "NONE"
                # here you can adjust which features you need in your lemmatization file !!!!!
                # outfile.write(f"{token.text}\t{token.pos_}\t{token.morph}\t{token.lemma_}\n")
                outfile.write(f"{token.text}\t{token.pos_}\t{token.lemma_}\n")

    return


### command to lemmatize the file

In [ ]:
parseFile('evaluation-set-v01.txt', 'evaluation-set-v01.vert', nlp_hy)

In [ ]:
parseFile('/content/TED2020-dehy-hy-aa.txt', '/content/TED2020-dehy-hy-aa--lemmatization-v01.txt', nlp_hy)